In [1]:
import pandas as pd
import numpy as np
import spacy
import string
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import gensim.downloader as api
from gensim.models import KeyedVectors

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
fake = pd.read_csv("/content/drive/MyDrive/NLP FAKE NEWS DETECTION/nlp dfnds/Fake.csv")
true = pd.read_csv("/content/drive/MyDrive/NLP FAKE NEWS DETECTION/nlp dfnds/True.csv")

In [9]:
fake['category']=1
true['category']=0

df=pd.concat([fake,true]).reset_index(drop=True)

In [10]:
# Separate the majority and minority classes
df_majority = df[df['category'] == 1]
df_minority = df[df['category'] == 0]

# Sample 3000 data points from each class
df_majority_sampled = df_majority.sample(n=3000, random_state=42)
df_minority_sampled = df_minority.sample(n=3000, random_state=42)

# Combine the sampled data
df_balanced = pd.concat([df_majority_sampled, df_minority_sampled])

# Shuffle the dataframe
df= df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the balanced dataset if needed
# df_balanced.to_csv('balanced_dataset.csv', index=False)

df_majority = df[df['category'] == 1]
df_minority = df[df['category'] == 0]

# Undersample the majority class
df_majority_undersampled = df_majority.sample(len(df_minority), random_state=42)

# Combine the undersampled majority class with the minority class
df = pd.concat([df_majority_undersampled, df_minority])

In [11]:
df=df[['text','category']]

In [12]:
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
punctuations = string.punctuation

In [13]:
model=api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [14]:
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [word.lemma_.lower().strip() for word in doc]

    # Removing stop words and punctuations
    mytokens = [word for word in mytokens if word not in stop_words and word not in punctuations]

    # Return preprocessed list of tokens
    return mytokens

In [15]:
df['tokens'] = df['text'].apply(spacy_tokenizer)

KeyboardInterrupt: 

In [ ]:
# Generate document vectors for each document
def document_vector(tokens, embeddings, dim):
    token_vectors = [embeddings[token] for token in tokens if token in embeddings]
    if not token_vectors:
        return np.zeros(dim)
    return np.mean(token_vectors, axis=0)

In [ ]:
df['vec'] = df['tokens'].apply(lambda x: document_vector(x, model, model.vector_size))

In [ ]:
# Prepare features (X) and labels (y)
X = np.vstack(df['vec'])
y = df['category']

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)

In [ ]:
model_pipeline_lr = Pipeline([
    ('lr', LogisticRegression(max_iter=1000, random_state=1))  # Increase max_iter if needed
])


In [ ]:
# Train the model
model_pipeline_lr.fit(X_train, y_train)

# Predict on the test set
y_pred_lr = model_pipeline_lr.predict(X_test)


In [ ]:
# Evaluate the model
accuracy_lr = accuracy_score(y_test, y_pred_lr)
precision_lr = precision_score(y_test, y_pred_lr, average='weighted')
recall_lr = recall_score(y_test, y_pred_lr, average='weighted')
f1_score_lr = f1_score(y_test, y_pred_lr, average='weighted')
classification_report_lr = classification_report(y_test, y_pred_lr)

# Print evaluation metrics
print("Evaluation Metrics for Logistic Regression Model")
print("------------------------------------------------")
print(f"Accuracy: {accuracy_lr:.4f}")
print(f"Precision: {precision_lr:.4f}")
print(f"Recall: {recall_lr:.4f}")
print(f"F1-score: {f1_score_lr:.4f}")
print("Classification Report:")
print(classification_report_lr)

Evaluation Metrics for Logistic Regression Model
------------------------------------------------
Accuracy: 0.9317
Precision: 0.9318
Recall: 0.9317
F1-score: 0.9317
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.94      0.93       592
           1       0.94      0.92      0.93       608

    accuracy                           0.93      1200
   macro avg       0.93      0.93      0.93      1200
weighted avg       0.93      0.93      0.93      1200

